In [ ]:
from sklearn.ensemble import RandomForestClassifier
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory

def main():
    # Add arguments to script
    parser = argparse.ArgumentParser()

    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")

    args = parser.parse_args()

    run = Run.get_context()

    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))

    # Create TabularDataset using TabularDatasetFactory
    
    from azureml.core.dataset import Dataset
    ds = Dataset.Tabular.from_delimited_files(path='https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data')
    
    # Split the data into train and test sets

    x = ds.to_pandas_dataframe()
    y = x["class"]
    x=x.drop['class', axis=1]
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

    ### YOUR CODE HERE end ################################################################################

    model = RandomForestClassifier(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    accuracy = model.score(x_test, y_test)
    run.log("accuracy", np.float(accuracy))
    
    ### MY CODE start w/o TODO instraction #################################################################
    import joblib
    os.makedirs('outputs', exist_ok=True)
    joblib.dump(model, 'outputs/hyperdrive_model.joblib')
    ### MY CODE end ########################################################################################

if __name__ == '__main__':
    main()